In [0]:
%pip install azure-identity azure-keyvault-secrets

In [0]:
%restart_python

In [0]:
from azure.identity import DefaultAzureCredential
from azure.keyvault.secrets import SecretClient

# storage mount with keyvault
key_vault_url = "https://anomalykeyvault10222025.vault.azure.net/"
key_name_tenant_id = "tenant-id"
key_name_client_id = "client-id"
key_name_client_secret = "client-secret"

# storage to mount
storge_account_name = "stlabelingdevwestus001"
container = "data"
mount_name = "datamount"

# keyvault credential
credential = DefaultAzureCredential()
client = SecretClient(vault_url=key_vault_url, credential=credential)

# get keyvault secret
tenant_id = client.get_secret(key_name_tenant_id).value
client_id = client.get_secret(key_name_client_id).value
client_secret = client.get_secret(key_name_client_secret).value

# OAuth and Mount
configs = {
    "fs.azure.account.auth.type": "OAuth",
    "fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
    "fs.azure.account.oauth2.client.id": client_id,
    "fs.azure.account.oauth2.client.secret": client_secret,
    "fs.azure.account.oauth2.client.endpoint": f"https://login.microsoftonline.com/{tenant_id}/oauth2/token"
}

# (참고) 이미 마운트된 경우 오류가 발생할 수 있으므로, 
# 재실행 시 dbutils.fs.unmount(f"/mnt/{mount_name}") 를 먼저 실행
try:
    dbutils.fs.mount(
        source = f"abfss://{container}@{storge_account_name}.dfs.core.windows.net/",
        mount_point = f"/mnt/{mount_name}",
        extra_configs = configs
    )
    print(f"Mount success: /mnt/{mount_name}")
except Exception as e:
    print(f"Error: {e}")
    print(f"/mnt/{mount_name} is already mounted or another error occurred.")

In [0]:
# list the contents of the mounted directory
dbutils.fs.ls(f"/mnt/{mount_name}")

In [0]:
dbutils.fs.ls(f"/mnt/{mount_name}/labels")